# Classification of Spoken Letters with the ISOLET Dataset

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from darwin.networks import EchoStateNetwork
from darwin.activations import sigmoid_activation

from qkeras.quantizers import quantized_relu

## Pre-processing of the ISOLET Dataset

In [2]:
# Download the training and validation ISOLET datasets from https://archive.ics.uci.edu/ml/machine-learning-databases/isolet/
# Store somewhere on your system, and include their path in the read_csv instructions below.

df = pd.read_csv('../sandbox/datasets/isolet1+2+3+4.data', sep=',', header=0)
X_train = df[df.columns[:-1]].to_numpy()
y_train = df[df.columns[-1]].to_numpy() - 1

df = pd.read_csv('../sandbox/datasets/isolet5.data', sep=',', header=0)
X_test = df[df.columns[:-1]].to_numpy()
y_test = df[df.columns[-1]].to_numpy() - 1

num_classes = 26
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

## Build the Binary ESN Model

In [3]:
nodes = 300
esn = EchoStateNetwork(
    n_inputs=X_train.shape[1],
    n_outputs=num_classes,
    n_neurons=nodes,
    sparsity=0.6,
    spectral_radius=None,
    activation=sigmoid_activation,
    nbit_quant=6,
    readout_layer= {
        'type': 'MultiLayerDense',
        'hyper_params': {
            'layers': [
                {'units': 60, 'activation': quantized_relu},
                {'units': num_classes, 'activation': 'softmax'}
            ],
            'optimizer': tf.keras.optimizers.Adadelta(lr=0.5),
            'loss': tf.keras.losses.categorical_crossentropy,
            'metrics': ['accuracy'],
        }
    },
    seed=0
)

/Users/valerio/miniconda3/envs/test/lib/python3.9/site-packages/keras/optimizer_v2/adadelta.py:74: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adadelta, self).__init__(name, **kwargs)
2022-01-27 17:25:03.052446: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Train the Readout Layer

In [4]:
history = esn.fit(
    X_train=X_train,
    y_train=y_train,
    X_val=X_test,
    y_val=y_test,
    batch_size=64,
    epochs=20
)

Epoch 1/20
98/98 [==============================] - 1s 6ms/step - loss: 2.9193 - accuracy: 0.3083 - val_loss: 2.4468 - val_accuracy: 0.5379
Epoch 2/20
98/98 [==============================] - 0s 2ms/step - loss: 2.0850 - accuracy: 0.6431 - val_loss: 1.7977 - val_accuracy: 0.7150
Epoch 3/20
98/98 [==============================] - 0s 2ms/step - loss: 1.5715 - accuracy: 0.7677 - val_loss: 1.4201 - val_accuracy: 0.7734
Epoch 4/20
98/98 [==============================] - 0s 3ms/step - loss: 1.2605 - accuracy: 0.8207 - val_loss: 1.1844 - val_accuracy: 0.8286
Epoch 5/20
98/98 [==============================] - 0s 3ms/step - loss: 1.0584 - accuracy: 0.8430 - val_loss: 1.0307 - val_accuracy: 0.8370
Epoch 6/20
98/98 [==============================] - 0s 2ms/step - loss: 0.9129 - accuracy: 0.8727 - val_loss: 0.8967 - val_accuracy: 0.8460
Epoch 7/20
98/98 [==============================] - 0s 2ms/step - loss: 0.8118 - accuracy: 0.8823 - val_loss: 0.8190 - val_accuracy: 0.8607
Epoch 8/20
98/98 [==

## Print Validation Accuracy

In [5]:
val_accuracy = history.history['val_accuracy'][-1] * 100
print(f'Validation accuracy: {val_accuracy:.2f}%')

Validation accuracy: 90.24%


## Show the Summary Table

In [6]:
esn.summary()

Model: Echo State Network
________________________________________________________________________________
Layer                               Neurons #                       Params #    
Reservoir                           300                             120         
________________________________________________________________________________


Model: "sequential"
________________________________________________________________________________
 Layer (type)                       Output Shape                    Param #     
 q_dense (QDense)                   (None, 60)                      18060       
                                                                                
 q_activation (QActivation)         (None, 60)                      0           
                                                                                
 q_dense_1 (QDense)                 (None, 26)                      1586        
                                                             